# Prepare a protein system from scratch

- BAT.py requires a protein system in PDB format. The atom names should be generic or match the names in the AMBER force field; PDB files from e.g. a snapshot from a CHARMM simulation, the output PDB from `dabble` when `charmm` forcefield is used, will not work.

We need two PDB files for preparing a protein system.

1. protein_input.pdb: A PDB of protein that is exported from Maestro; it means the protonation states of the protein are assigned (I think tleap can recongnize e.g ASP and ASH, maybe?). Water and ligand can be present in the protein_input.pdb but they will be removed during preparation.

2. system_input.pdb: A prepared simulation system with dabble. The ligand does not need to be present in the system_input.pdb.

For ligands that will be used in the simulation, we need their PDB files as well. The ligand should be in the docking poses for `system_input.pdb`.

3. system_input.inpcrd: The inpcrd file of the system_input.pdb
This is used to get the dimension of the box.

To get the anchor atoms for the protein, prepare a PDB with ligand docked into the protein `prot_lig_input.pdb`; it should share the same resid as the protein in `protein_input.pdb`. The ligand should be in the docking pose.

In [4]:
import MDAnalysis as mda
import numpy as np
from MDAnalysis.analysis import align

In [5]:
protein_input = 'protein_input.pdb'
system_input = 'system_input.pdb'
system_inpcrd = 'system_input.inpcrd'
prot_lig_input = 'prot_lig_input.pdb'

In [6]:
u_prot = mda.Universe(protein_input)
u_sys = mda.Universe(system_input, format='XPDB')
u_prot_lig = mda.Universe(prot_lig_input)

## Generate the protein system

1. Add POPC from system_input.pdb back to protein_input.pdb**
2. Store water that are within 20 Å from the protein/membrane system

In [10]:
u_sys_aligned = mda.Universe(system_input, format = 'XPDB')
align.alignto(u_sys_aligned,
              u_prot, select='name CA and resid 60 to 250', match_atoms=False)

/scratch/users/yuzhuang/miniforge3/envs/batter/lib/python3.11/site-packages/MDAnalysis/topology/PDBParser.py:295: UserWarning: PDB file is missing resid information.  Defaulted to '1'
  warnings.warn("PDB file is missing resid information.  "
/scratch/users/yuzhuang/miniforge3/envs/batter/lib/python3.11/site-packages/MDAnalysis/topology/PDBParser.py:346: UserWarning: Unknown element  found for some atoms. These have been given an empty element record. If needed they can be guessed using universe.guess_TopologyAttrs(context='default', to_guess=['elements']).
  warnings.warn(wmsg)


(0.0005844909808033118, 0.0005838093667875604)

In [11]:
membrane_ag = u_sys_aligned.select_atoms('resname POPC')
print(f'Number of POPC molecules: {membrane_ag.n_residues}')

Number of POPC molecules: 171


In [12]:
water_ag = u_sys_aligned.select_atoms('byres (resname TIP3 and around 20 (protein or resname POPC))')
print(f'Number of water molecules: {water_ag.n_residues}')

Number of water molecules: 13992


In [14]:
u_merged = mda.Merge(u_prot.select_atoms('protein'), membrane_ag, water_ag)

In [16]:
water = u_merged.select_atoms('resname TIP3')
print(f'Number of water molecules: {water.n_residues}')
print(f'Water atom names: {water.residues[0].atoms.names}')

Number of water molecules: 13992
Water atom names: ['OH2' 'H1' 'H2']


In [18]:
# set OW, OH2 to O
# Otherwise tleap cannot recognize the water molecules
water.select_atoms('name OW').names = 'O'
water.select_atoms('name OH2').names = 'O'

In [19]:
# read last line of inpcrd file to get dimensions
with open(system_inpcrd) as f:
    lines = f.readlines()
    box = np.array([float(x) for x in lines[-1].split()])
    print(f'Box dimensions: {box}')

u_merged.dimensions = box

Box dimensions: [ 89.935204  85.773499 112.995003  90.        90.        90.      ]


In [20]:
# save as *_docked.pdb that matched `input-dd-amber.in`
u_merged.atoms.write('MOR_docked.pdb')

/scratch/users/yuzhuang/miniforge3/envs/batter/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


## Generate reference structure

In [21]:
protein_ref = u_sys.select_atoms('protein')

In [22]:
protein_ref.write('../build_files/reference.pdb')

/scratch/users/yuzhuang/miniforge3/envs/batter/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "


## Get protein and ligand anchors
Follow the guideline in section 7 of https://github.com/GHeinzelmann/BAT.py/blob/master/doc/User-guide.pdf

Visualize and select anchor atoms with VMD.

Save the final l1_x,y,z values inside `input-dd-amber.in`

In [23]:
P1_atom = u_prot_lig.select_atoms('name CA and resid 149')
P2_atom = u_prot_lig.select_atoms('name CA and resid 119')
P3_atom = u_prot_lig.select_atoms('name CA and resid 328')
if P1_atom.n_atoms != 1 or P2_atom.n_atoms != 1 or P3_atom.n_atoms != 1:
    raise ValueError('Error: more than one atom selected')

In [24]:
potential_lig_l1 = u_prot_lig.select_atoms('resname MP and name C12')
if potential_lig_l1.n_atoms != 1:
    raise ValueError('There should be exactly one atom named C12 in the ligand')

In [25]:
# get ll_x,y,z distances

r_vect = potential_lig_l1.positions - P1_atom.positions
print(f'l1_x: {r_vect[0][0]:.2f}')
print(f'l1_y: {r_vect[0][1]:.2f}')
print(f'l1_z: {r_vect[0][2]:.2f}')

l1_x: 2.08
l1_y: -6.83
l1_z: 3.94
